In [2]:
import pyodbc
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# Verbindungsstring zur MS SQL-Datenbank
conn_str = (
    r'Driver={SQL Server};'
    r'Server=.\SQLEXPRESS;'
    r'Database=studienprojekt;'
    r'Trusted_Connection=yes;'
)

# Verbindung zur MS SQL-Datenbank herstellen und Daten abrufen
connection = pyodbc.connect(conn_str)
query = "SELECT timestamp, o, c, h, l FROM stock_data ORDER BY timestamp"
data = pd.read_sql(query, connection)

# Daten vorbereiten
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data[['o', 'c', 'h', 'l']])

# Fenstergröße für die Sequenzen
window_size = 30

# Erzeuge Trainingsdaten und Testdaten
train_data = scaled_data[:-window_size]
test_data = scaled_data[-window_size:]

# Funktion zum Erzeugen von Sequenzen
def create_sequences(data, window_size):
    X = []
    y = []
    for i in range(len(data) - window_size):
        X.append(data[i:i + window_size])
        y.append(data[i + window_size])
    return np.array(X), np.array(y)

# Erzeuge Sequenzen für das Training
X_train, y_train = create_sequences(train_data, window_size)

# LSTM-Modell erstellen und trainieren
model = Sequential()
model.add(LSTM(64, activation='relu', input_shape=(window_size, 4)))
model.add(Dense(4))
model.compile(optimizer='adam', loss='mse')
model.fit(X_train, y_train, epochs=100, batch_size=16)

# Vorhersagen für den nächsten Monat generieren
last_data = test_data[-window_size:]
predictions = []
for _ in range(30):
    input_data = np.reshape(last_data, (1, window_size, 4))
    prediction = model.predict(input_data)
    predictions.append(prediction[0])
    last_data = np.append(last_data[1:], prediction[0])

# Skaliere die Vorhersagen in den ursprünglichen Wertebereich zurück
predictions = scaler.inverse_transform(predictions)

# Ergebnisse anzeigen
print("Vorhersagen für den nächsten Monat:")
for i, prediction in enumerate(predictions):
    print(f"Tag {i+1}: {prediction}")


C:\Users\HerbyHerb\AppData\Local\Temp\ipykernel_9380\4144051015.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data = pd.read_sql(query, connection)


Epoch 1/100
28/28 [==============================] - 3s 13ms/step - loss: 0.1311
Epoch 2/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0124
Epoch 3/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0078
Epoch 4/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0061
Epoch 5/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0054
Epoch 6/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0051
Epoch 7/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0048
Epoch 8/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0047
Epoch 9/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0047
Epoch 10/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0043
Epoch 11/100
28/28 [==============================] - 0s 13ms/step - loss: 0.0040
Epoch 12/100
28/28 [==============================] - 0s 14ms/step - loss: 0.0039
Epoch 13/100
28/28 [=====

1/1 [==============================] - 0s 20ms/step


ValueError: cannot reshape array of size 123 into shape (1,30,4)